In [ ]:
import obspy
import glob
import matplotlib.pyplot as plt
import os
import shutil
import subprocess

# Set up all paths that will be used
homedir = os.path.expanduser('~')
softwaredir = os.path.join(homedir, 'Desktop', 'software')
WINSUDSPATH = os.path.join(softwaredir,'winsuds','bin')
datatop = os.path.join(homedir, 'Desktop', 'DATA')
datadir = os.path.join(datatop, 'NevadoDelRuiz','suds')#,'2012', '04', '04')
demux = os.path.join(WINSUDSPATH, 'demux.exe')
irig = os.path.join(WINSUDSPATH, 'irig.exe')
sud2sac = os.path.join(WINSUDSPATH, 'sud2sac.exe')
sud2msed = os.path.join(WINSUDSPATH, 'sud2msed.exe')
"""
allwvmfiles = sorted(glob.glob(os.path.join(datadir, '*.WVR')))
print(f'got {len(allwvmfiles)} files')
print(datadir)
originalfile = allwvmfiles[9]
os.chdir(datatop)
#newfile = os.path.basename(originalfile)
newfile = os.path.join(datatop, os.path.basename(originalfile))
shutil.copy(originalfile, newfile)
#SUDSbasename = os.path.basename(newfile).replace('.WVR','')
SUDSbasename = newfile.replace('.WVR','')
print(SUDSbasename)
WVMfile = SUDSbasename + '.WVR' # the original multiplexed PC-SUDS format event waveform file
DMXfile = SUDSbasename + '.DMX' # produced by demux.exe and irig.exe
SACbasename = SUDSbasename + '.sac' # produced  by sud2sac.exe
MSEEDfile = SUDSbasename + '.mseed' # produced by recombining SAC file
"""

def runCommand(cmdlist):
    print(' '.join(cmdlist))
    result = subprocess.run(cmdlist, capture_output=True, text=True)
    print(result.stdout)
    
#### MAIN PROGRAM STARTS HERE #############
# Loop over year directories
yyyydirs = sorted(glob.glob(os.path.join(datadir, '[1-2][0-9][0-9][0-9]')))
for yyyydir in yyyydirs:
    #yyyy = os.path.basename(yyyydir)
    mmdirs = sorted(glob.glob(os.path.join(yyyydir,'[0-1][0-9]')))
    for mmdir in mmdirs:
        #mm = os.path.basename(mmdir)
        dddirs = sorted(glob.glob(os.path.join(mmdir, '[0-1][0-9]')))
        for dddir in dddirs:
            WVMfiles = sorted(glob.glob(os.path.join(dddir, '*.WVR')))
            print('Found %d WVM files in %s' % (len(WVMfiles), dddir))
            for WVMfile in WVMfiles:
                SUDSdirbase = WVMfile[0:-4]
                SUDSbasename = os.path.basename(SUDSdirbase)
                MSEEDfile = SUDSdirbase + '.mseed' # produced by recombining SAC file
                DMXfile = SUDSdirbase + '.dmx'
                print('Demultiplexing ' + WVMfile)
                runCommand([demux, WVMfile])
                if (os.path.exists(DMXfile)):
                    print('- Success')
                else:
                    print('Demultiplexing seems to have failed as ' + DMXfile + ' not created. Adding to list of bad WVM files.')
                    fbad = open('badWVMfileList.txt','a+')
                    fbad.write(WVMfile + '\n')
                    fbad.close()
                    continue

                print('Time correcting ' + DMXfile)
                runCommand([irig, DMXfile])

                # This produces one file per trace
                print('Converting ' + DMXfile + ' to SAC files')
                try:
                    runCommand([sud2sac, DMXfile])
                    os.remove(DMXfile)
                except:
                    print('Crashed on converting')
                SACdirbase = SUDSdirbase + '.sac' # produced  by sud2sac.exeyy
                sacfilelist = glob.glob(SACdirbase + '-???')

                # Now merge the SAC files into a single valid Miniseed file    
                if len(sacfilelist) > 0:
                    st = obspy.Stream()
                    for sacfile in sacfilelist:
                        print('Combining ' + sacfile + ' into Miniseed file ' + MSEEDfile)
                        try:
                            sacst = obspy.read(sacfile)
                            st = st + sacst
                        except:
                            pass
                        os.remove(sacfile)
                    if len(st) == 0:
                        print('No good SAC files')
                        continue
                    st.write(MSEEDfile)

                else:
                    print('No SAC files found matching ' + SACdirbase + '*. Must have been bad DMX file ' + DMXfile)
                    fbad = open('badDMXfileList.txt','a+')
                    fbad.write(DMXfile + '\n')
                    fbad.close()

Found 720 WVM files in C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01
Demultiplexing C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010000.WVR
C:\Users\Glenn Thompson\Desktop\software\winsuds\bin\demux.exe C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010000.WVR
DEMUX - Win32 Version 2.90

Input file:  C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010000.WVR
Output file: C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010000.dmx

Copying structures
Demuxing channel: 1 
Demuxing channel: 2 
Demuxing channel: 3 
Demuxing channel: 4 
Demuxing channel: 5 
Demuxing channel: 6 
Demuxing channel: 7 
Demuxing channel: 8 
Demuxing channel: 9 
Demuxing channel: 10 
Demuxing channel: 11 
Demuxing channel: 12 
Demuxing channel: 13 
Demuxing channel: 14 
Demuxing channel: 15 
Demuxing channel: 16 
Demuxing channel: 17 
Demuxing channel: 18 
Demuxing channel: 19 
Demuxing channel: 20 
Demuxin

SUDS to SAC data file conversion - Version Win32 1.40

Indexing input file: C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.dmx

STN:IRIG, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-000
STN:OLLZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-001
STN:OLLN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-002
STN:OLLE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-003
STN:BISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-004
STN:BISN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-005
STN:BISE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-006
STN:LISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010002.sac-007
STN:LISE, CHN:0 -> 

- Success
Time correcting C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010004.dmx
C:\Users\Glenn Thompson\Desktop\software\winsuds\bin\irig.exe C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010004.dmx
IRIG-E decoder - Win32 Version 2.90
Time code channel: IRIG
Time code channel: IRIG

C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010004.dmx corrected on 12/07/24 21:31:43.463
Corrected IST:  04/01/12 00:04:42.511
Delta T:        -18005.488638
Corrected rate: 100.018182
Delta rate      +0.018182
Frame 1: Quality=4, Lock=Yes, Time=04/01/12 00:04:50.000
Frame 2: Quality=4, Lock=Yes, Time=04/01/12 00:05:00.000
Frame 3: Quality=4, Lock=Yes, Time=04/01/12 00:05:10.000
Frame 4: Quality=4, Lock=Yes, Time=04/01/12 00:05:20.000
Frame 5: Quality=4, Lock=Yes, Time=04/01/12 00:05:30.000
Frame 6: Quality=4, Lock=Yes, Time=04/01/12 00:05:40.000
Frame 7: Quality=4, Lock=Yes, Time=04/01/12 00:05:50.000
Frame 8: Quality=4, Lock=Yes, Ti

SUDS to SAC data file conversion - Version Win32 1.40

Indexing input file: C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.dmx

STN:IRIG, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-000
STN:OLLZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-001
STN:OLLN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-002
STN:OLLE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-003
STN:BISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-004
STN:BISN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-005
STN:BISE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-006
STN:LISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010006.sac-007
STN:LISE, CHN:0 -> 

SUDS to SAC data file conversion - Version Win32 1.40

Indexing input file: C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.dmx

STN:IRIG, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-000
STN:OLLZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-001
STN:OLLN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-002
STN:OLLE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-003
STN:BISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-004
STN:BISN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-005
STN:BISE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-006
STN:LISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010008.sac-007
STN:LISE, CHN:0 -> 

...

- Success
Time correcting C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010836.dmx
C:\Users\Glenn Thompson\Desktop\software\winsuds\bin\irig.exe C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010836.dmx
IRIG-E decoder - Win32 Version 2.90
Time code channel: IRIG
Time code channel: IRIG

C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010836.dmx corrected on 12/07/24 21:33:06.147
Corrected IST:  04/01/12 08:36:38.180
Delta T:        -18009.819669
Corrected rate: 100.018182
Delta rate      +0.018182
Frame 1: Quality=4, Lock=Yes, Time=04/01/12 08:36:40.000
Frame 2: Quality=4, Lock=Yes, Time=04/01/12 08:36:50.000
Frame 3: Quality=4, Lock=Yes, Time=04/01/12 08:37:00.000
Frame 4: Quality=4, Lock=Yes, Time=04/01/12 08:37:10.000
Frame 5: Quality=4, Lock=Yes, Time=04/01/12 08:37:20.000
Frame 6: Quality=4, Lock=Yes, Time=04/01/12 08:37:30.000
Frame 7: Quality=4, Lock=Yes, Time=04/01/12 08:37:40.000
Frame 8: Quality=4, Lock=Yes, Ti

SUDS to SAC data file conversion - Version Win32 1.40

Indexing input file: C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.dmx

STN:IRIG, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-000
STN:OLLZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-001
STN:OLLN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-002
STN:OLLE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-003
STN:BISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-004
STN:BISN, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-005
STN:BISE, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-006
STN:LISZ, CHN:0 -> C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010838.sac-007
STN:LISE, CHN:0 -> 

DEMUX - Win32 Version 2.90

Input file:  C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010840.WVR
Output file: C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010840.dmx

Copying structures
Demuxing channel: 1 
Demuxing channel: 2 
Demuxing channel: 3 
Demuxing channel: 4 
Demuxing channel: 5 
Demuxing channel: 6 
Demuxing channel: 7 
Demuxing channel: 8 
Demuxing channel: 9 
Demuxing channel: 10 
Demuxing channel: 11 
Demuxing channel: 12 
Demuxing channel: 13 
Demuxing channel: 14 
Demuxing channel: 15 
Demuxing channel: 16 
Demuxing channel: 17 
Demuxing channel: 18 
Demuxing channel: 19 
Demuxing channel: 20 
Demuxing channel: 21 
Demuxing channel: 22 
Demuxing channel: 23 
Demuxing channel: 24 
Demuxing channel: 25 
Demuxing channel: 26 
Demuxing channel: 27 
Demuxing channel: 28 
Demuxing channel: 29 
- Success
Time correcting C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010840.dmx
C:\Users\Glenn Thompson\Desktop

- Success
Time correcting C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010842.dmx
C:\Users\Glenn Thompson\Desktop\software\winsuds\bin\irig.exe C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010842.dmx
IRIG-E decoder - Win32 Version 2.90
Time code channel: IRIG
Time code channel: IRIG

C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010842.dmx corrected on 12/07/24 21:33:07.191
Corrected IST:  04/01/12 08:42:38.130
Delta T:        -18009.869660
Corrected rate: 100.018182
Delta rate      +0.018182
Frame 1: Quality=4, Lock=Yes, Time=04/01/12 08:42:40.000
Frame 2: Quality=4, Lock=Yes, Time=04/01/12 08:42:50.000
Frame 3: Quality=4, Lock=Yes, Time=04/01/12 08:43:00.000
Frame 4: Quality=4, Lock=Yes, Time=04/01/12 08:43:10.000
Frame 5: Quality=4, Lock=Yes, Time=04/01/12 08:43:20.000
Frame 6: Quality=4, Lock=Yes, Time=04/01/12 08:43:30.000
Frame 7: Quality=4, Lock=Yes, Time=04/01/12 08:43:40.000
Frame 8: Quality=4, Lock=Yes, Ti

IRIG-E decoder - Win32 Version 2.90
Time code channel: IRIG
Time code channel: IRIG

C:\Users\Glenn Thompson\Desktop\DATA\NevadoDelRuiz\suds\2012\04\01\04010844.dmx corrected on 12/07/24 21:33:07.556
Corrected IST:  04/01/12 08:44:38.110
Delta T:        -18009.889657
Corrected rate: 100.018182
Delta rate      +0.018182
Frame 1: Quality=4, Lock=Yes, Time=04/01/12 08:44:40.000
Frame 2: Quality=4, Lock=Yes, Time=04/01/12 08:44:50.000
Frame 3: Quality=4, Lock=Yes, Time=04/01/12 08:45:00.000
Frame 4: Quality=4, Lock=Yes, Time=04/01/12 08:45:10.000
Frame 5: Quality=4, Lock=Yes, Time=04/01/12 08:45:20.000
Frame 6: Quality=4, Lock=Yes, Time=04/01/12 08:45:30.000
Frame 7: Quality=4, Lock=Yes, Time=04/01/12 08:45:40.000
Frame 8: Quality=4, Lock=Yes, Time=04/01/12 08:45:50.000
Frame 9: Quality=4, Lock=Yes, Time=04/01/12 08:46:00.000
Frame 10: Quality=4, Lock=Yes, Time=04/01/12 08:46:10.000
Frame 11: Quality=4, Lock=Yes, Time=04/01/12 08:46:20.000

Converting C:\Users\Glenn Thompson\Desktop\DATA\N